# Accessing the pulse implementation of native gates
All gates are implemented with a sequence of pulses that is carefully calibrated by quantum hardware providers. In this notebook, we will show how to:
- retrieve the calibrations for any particular native gate of interest, 
- modify it 
- submit a circuit where we will overwrite the provider's calibrations and use our modified ones.

These calibrations will be vended as `PulseSequence` objects and will contain a list of low-level instructions that are played in place of the gate that you inserted. Obtaining the accurate sequence that used by the provider offers many opportunities. First, it is one of the best way to understand how quantum computers works at the most fundamental layer. It also allows you to improve the results of your circuits by deploying error mitigation techniques such as zero noise extrapolation by stretching all the pulse of specific gates.

Native gate calibrations are available with Rigetti's Aspen M-3.

First, we will start by initializing a cost tracker and import some libraries.

In [1]:
# Use Braket SDK Cost Tracking to estimate the cost to run this example
from braket.tracking import Tracker
t = Tracker().start()

In [2]:
%matplotlib inline

from braket.aws import AwsDevice
from braket.native_gates import NativeGateCalibration
from braket.parametric import FreeParameter
from braket.circuits import Circuit, QubitSet
from braket.circuits.gates import Rx, XY
from braket.circuits.serialization import IRType

import math
from utils.draw_pulse_sequence import draw
import matplotlib.pyplot as plt

Calibrations are associated with a device so we will need to instantiate an AWS device. By default, the device properties are not automatically populated to limit the amount of data downloaded.

In [3]:
device = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-3")

# Test if the calibrations have been downloaded
print(device._native_gate_calibration is None)

True


Calibrations are accessible via the `native_gate_calibration` field of the device object. When accessed for the first time, the latest calibrations for all the gates will be downloaded. This takes between XX and YY seconds depending on the performance of your internet connection. The calibrations will be then cached as a dictionnary that is indexed by a tuple formed from a gate object and the qubit number.

In [10]:
calibrations = device.native_gate_calibration
print(f"We have downloaded {len(calibrations.calibration_data)} calibrations.")

We have downloaded 929 calibrations.


To refresh the calibration cache, you can run the following method

In [11]:
device.refresh_native_gate_calibrations()

Each calibration is represented by a `PulseSequence` object that contains the OpenPulse implementation for the associated native gate. Retrieving a gate in particular can be done by passing the `Gate` object (instantiating with a numerical value or a `FreeParameter` if necessary) and the qubit of interest. For instance, you can inspect the pulse implementation of the RX($\pi$/2) with the code in next cell.
As you have access to a `PulseSequence` object, you can generate the time trace of this pulse sequence via the `.to_time_trace()` and visualization with a plotting library like `matplotlib`. We use here a `draw` method that has been in placed in the python file at the root of this notebook to create the plots. 

In [12]:
xy_theta = XY(FreeParameter("θ"))
rx_pi_2 = Rx(math.pi/2)

a=10
b=113
pulse_sequence_xy_theta_q10_q113 = calibrations.get_pulse_sequence([(xy_theta, [10, 113])])
print(pulse_sequence_xy_theta_q10_q113.to_ir())

pulse_sequence_rx_pi_2_q0 = calibrations.get_pulse_sequence([(rx_pi_2, 0)])
draw(pulse_sequence_rx_pi_2_q0)

TypeError: 'NativeGateCalibration' object is not subscriptable

The fidelity of each gate is reported by Rigetti. It is consultable on the Braket console and programmatically through the device properties:

In [66]:
# method 1
print(f"fidelity of the XY gate between qubits {a} and {b}: ", device.properties.provider.specs["2Q"][f"{a}-{b}"]["fXY"])

# method 2
print(f"fidelity of the XY gate between qubits {a} and {b}: ", calibrations.get_fidelity([(Rx(math.pi/2), 0)]))

fidelity of the XY gate between qubits 10 and 113:  0.8629862902518167


# Attaching calibrations to redefine gates

When submitting a circuit to Rigetti's Aspen M-3 via Braket, all the gates of your circuit will be compiled into native gates, which are ultimately replaced by these sequences of pulses. You can now attach custom-made `PulseSequence`s as calibrations to modify the default behavior.

In [17]:
bell_circuit = (
    Circuit()
    .rx(0,math.pi/2)
    .rx(1,math.pi/2)
    .cz(0,1)
    .rx(1,-math.pi/2)
)
print(bell_circuit)

T  : |   0    |1|    2    |
                           
q0 : -Rx(1.57)-C-----------
               |           
q1 : -Rx(1.57)-Z-Rx(-1.57)-

T  : |   0    |1|    2    |


In [18]:
print(bell_circuit.to_ir(IRType.OPENQASM).source)

OPENQASM 3.0;
bit[2] b;
qubit[2] q;
rx(1.5707963267948966) q[0];
rx(1.5707963267948966) q[1];
cz q[0], q[1];
rx(-1.5707963267948966) q[1];
b[0] = measure q[0];
b[1] = measure q[1];


In [ ]:
nb_shots = 50
task=device.run(bell_circuit, shots=nb_shots)

counts = task.result().measurement_counts
plt.bar(sorted(counts), [counts[k]/nb_shots for k in sorted(counts)])
plt.xlabel("State")
plt.ylabel("Population")

Attaching the calibrations that we have downloaded in one of the first cell of this notebook, we can submit again the same circuit and see that we do not have significant changes compared to the first circuit execution

In [5]:
custom_calibration = NativeGateCalibration({(Rx(math.pi/2), QubitSet(0)): pulse_sequence_rx_pi_2_q0})
task=device.run(bell_circuit, native_gate_calibration=custom_calibration, shots=nb_shots)

counts = task.result().measurement_counts
plt.bar(sorted(counts), [counts[k]/nb_shots for k in sorted(counts)])
plt.xlabel("State")
plt.ylabel("Population")

T  : |   0    |1|
                 
q0 : -Rx(1.57)-C-
               | 
q1 : ----------X-

T  : |   0    |1|


If we now modify the pulse sequence such that the evolution achieved by RX($\pi$/2) is now half a rotation along the X axis (i.e. RX($\pi$)), then the outcome of the bell_circuit will be totally different. We expect in this case that the output state is $|11\rangle$.

In [ ]:
(gaussian_waveform_name, gaussian_waveform), = pulse_sequence_rx_pi_2_q0._waveform.items()
pulse_sequence_rx_pi_2_q0._waveform[gaussian_waveform_name].sigma = gaussian_waveform.sigma * 2
pulse_sequence_rx_pi_2_q0._waveform[gaussian_waveform_name].length = gaussian_waveform.length * 2

task=device.run(bell_circuit, native_gate_calibration=NativeGateCalibration({(Rx(math.pi/2), QubitSet(0)): pulse_sequence_rx_pi_2_q0}), shots=nb_shots)

counts = task.result().measurement_counts
plt.bar(sorted(counts), [counts[k]/nb_shots for k in sorted(counts)])
plt.xlabel("State")
plt.ylabel("Population")

In [20]:
print("Task Summary")
print(t.quantum_tasks_statistics())
print('Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).')
print(f"Estimated cost to run this example: {t.qpu_tasks_cost() + t.simulator_tasks_cost():.3f} USD")

Task Summary
{'arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-3': {'shots': 12000, 'tasks': {'COMPLETED': 120}}}
Note: Charges shown are estimates based on your Amazon Braket simulator and quantum processing unit (QPU) task usage. Estimated charges shown may differ from your actual charges. Estimated charges do not factor in any discounts or credits, and you may experience additional charges based on your use of other services such as Amazon Elastic Compute Cloud (Amazon EC2).
Estimated cost to run this example: 40.200 USD
